<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оценка эффективности каналов привлечения (`acquisition_channel`)

## ROI

### Определение

**ROI (Return on Investment)** — это **окупаемость инвестиций**, то есть отношение **прибыли к затратам**.

$$
\text{ROI} = \frac{\text{Выручка} - \text{Расходы}}{\text{Расходы}} = \frac{\text{Прибыль}}{\text{Расходы}}
$$

Где:

* **Выручка** — сумма заказов, совершённых пользователями, привлечёнными через канал.
* **Расходы** — сумма затрат на привлечение этих пользователей (обычно CPI × кол-во установок).

### Интерпретация

| ROI        | Интерпретация                                                                   |
| ---------- | ------------------------------------------------------------------------------- |
| > 0        | Канал **окупается**: доход выше расходов                                        |
| = 0        | Канал **вышел в ноль**: доход = расходам                                        |
| < 0        | Канал **убыточен**: вы тратите больше, чем зарабатываете                        |
| > 1 (100%) | Прибыль в 2 раза больше, чем вложения (например, потратили 100₴, получили 200₴) |



### Пример

| Канал    | Установки | CPI (₴) | Расходы | Выручка | ROI                        |
| -------- | --------- | ------- | ------- | ------- | -------------------------- |
| Facebook | 100       | 20₴     | 2000₴   | 3000₴   | (3000−2000)/2000 = 0.5     |
| TikTok   | 100       | 30₴     | 3000₴   | 2000₴   | (2000−3000)/3000 = −0.33   |
| Organic  | 200       | 0₴      | 0₴      | 1500₴   | (1500−0)/0 = ∞ (бесконеч.) |

> У Organic ROI бесконечен, потому что расходов нет — это бесплатный канал

### Важные замечания

* ROI **зависит от периода времени**. Покупки могут происходить через дни/недели — важно учитывать это в расчётах.
* Важно **учитывать отложенную выручку** (не только первую покупку, но и повторные).
* Для точности лучше рассчитывать **ROI по когортам** (например, через 7, 30, 60 дней).

In [30]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [31]:
DB_USER = "da_test_user"
DB_PASS = "c89M2psBdXGtBXpY3BPrTRZ2bXl6yq2i"
DB_HOST = "dpg-d1hvnobe5dus739ipbcg-a.frankfurt-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "da_test"

In [32]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [54]:
query = """
SELECT
    a.acquisition_channel,
    COUNT(DISTINCT a.device_code) AS installs,
    SUM(a.cpi_uah) AS total_cost,
    ROUND(AVG(a.cpi_uah), 2) AS avg_cpi,
    COUNT(DISTINCT v.device_code) AS viewers,
    COUNT(DISTINCT o.user_uuid) AS buyers,
    SUM(o.total_uah) AS total_revenue

FROM app_sessions a

LEFT JOIN (
    SELECT DISTINCT device_code
    FROM product_views_log
) v ON v.device_code = a.device_code

LEFT JOIN (
    SELECT DISTINCT device_code, user_uuid
    FROM devices_users_map
) m ON m.device_code = a.device_code

LEFT JOIN (
    SELECT user_uuid, SUM(total_uah) AS total_uah
    FROM orders_log
    GROUP BY user_uuid
) o ON o.user_uuid = m.user_uuid

GROUP BY a.acquisition_channel
ORDER BY total_revenue DESC;
"""

In [55]:
df = pd.read_sql(text(query), engine)
df

,acquisition_channel,installs,total_cost,avg_cpi,viewers,buyers,total_revenue
0,Facebook,134,3401.53,25.38,134,89,1150615.53
1,Referral,132,3369.93,25.53,132,73,994129.97
2,Organic,111,2576.55,23.21,111,70,937531.37
3,Google Ads,123,3196.90,25.99,123,68,913383.78


In [47]:
df['roi'] = (df['total_revenue'] - df['total_cost']) / df['total_cost']
df

,acquisition_channel,installs,total_cost,avg_cpi,viewers,buyers,total_revenue,roi
0,Facebook,134,132440.79,26.40,134,89,7365475.53,54.613346
1,Referral,132,121128.89,25.79,132,73,6877018.62,55.774388
2,Organic,111,94708.77,22.51,111,70,6068311.12,63.073381
3,Google Ads,123,107237.99,25.83,123,68,5918501.76,54.190346


In [48]:
# CR — conversion rate в просмотры
df["cr_view"] = (df["viewers"] / df["installs"]).round(3)

# CR — conversion rate из установок в покупателей
df["cr_buyer"] = (df["buyers"] / df["installs"]).round(3)

In [49]:
df

,acquisition_channel,installs,total_cost,avg_cpi,viewers,buyers,total_revenue,roi,cr_view,cr_buyer
0,Facebook,134,132440.79,26.40,134,89,7365475.53,54.613346,1.0,0.664
1,Referral,132,121128.89,25.79,132,73,6877018.62,55.774388,1.0,0.553
2,Organic,111,94708.77,22.51,111,70,6068311.12,63.073381,1.0,0.631
3,Google Ads,123,107237.99,25.83,123,68,5918501.76,54.190346,1.0,0.553


In [38]:
df.sort_values("total_revenue", ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,acquisition_channel,installs,total_cost,avg_cpi,viewers,buyers,total_revenue,roi,cr_view,cr_buyer
0,Facebook,134,132440.79,26.40,134,89,7365475.53,54.613346,1.0,0.664
1,Referral,132,121128.89,25.79,132,73,6877018.62,55.774388,1.0,0.553
2,Organic,111,94708.77,22.51,111,70,6068311.12,63.073381,1.0,0.631
3,Google Ads,123,107237.99,25.83,123,68,5918501.76,54.190346,1.0,0.553


In [39]:
fig = px.bar(
    df,
    x="acquisition_channel",
    y="roi",
    color="roi",
    title="📊 ROI по каналам привлечения",
    text="roi",
    color_continuous_scale="RdYlGn"
)
fig.update_layout(yaxis_title="ROI", xaxis_title="Канал привлечения")
fig.show()

# Подзапросы и CTE в SQL

## Подзапрос (Subquery)

Подзапрос — это **запрос внутри другого запроса**. Он может стоять в:

* `SELECT`:
  → чтобы вернуть одно значение (скаляр)
* `FROM`:
  → как временная таблица
* `WHERE` / `IN` / `EXISTS`:
  → для фильтрации

### Пример подзапроса в `SELECT`

In [41]:
# Подсчитываем общее число заказов каждого пользователя, и отдельно — его средний чек:
query = """
SELECT
    user_uuid,
    COUNT(*) AS order_count,
    AVG(total_uah) AS avg_check1,
    (
      SELECT AVG(total_uah)
     FROM orders_log o2
     WHERE o2.user_uuid = o1.user_uuid) AS avg_check
FROM orders_log o1
GROUP BY user_uuid
LIMIT 10;
"""

df = pd.read_sql(text(query), engine)
df.head()

,user_uuid,order_count,avg_check1,avg_check
0,USR0107,5,989.436000,989.436000
1,USR0066,13,1466.497692,1466.497692
2,USR0106,14,1369.988571,1369.988571
3,USR0224,9,2032.513333,2032.513333
4,USR0138,14,1351.451429,1351.451429


### Пример подзапроса в `FROM`:

In [42]:
# Вычислим выручку по каждому пользователю в подзапросе, а снаружи отфильтруем тех, кто принёс более 500 грн
query = """
SELECT *
FROM (
    SELECT user_uuid, SUM(total_uah) AS revenue
    FROM orders_log
    GROUP BY user_uuid
) AS user_revenue
WHERE revenue > 500;
"""

df = pd.read_sql(text(query), engine)
df.head()

,user_uuid,revenue
0,USR0107,4947.18
1,USR0066,19064.47
2,USR0106,19179.84
3,USR0224,18292.62
4,USR0138,18920.32


### Пример подзапроса в `WHERE`:

In [43]:
query = """
SELECT DISTINCT user_uuid
FROM devices_users_map
WHERE user_uuid IN (
    SELECT DISTINCT user_uuid
    FROM orders_log
);
"""

df = pd.read_sql(text(query), engine)
df.head()

,user_uuid
0,USR0107
1,USR0066
2,USR0106
3,USR0224
4,USR0138


## CTE — Common Table Expression (выражение общей таблицы)

`WITH` или CTE — это способ **дать имя подзапросу**, чтобы потом использовать его как обычную таблицу **во внешнем запросе**.

### Синтаксис:

```sql
WITH имя_таблицы AS (
    SELECT ... -- тут подзапрос
)
SELECT ...
FROM имя_таблицы;
```

### Пример:

In [50]:
query = """
WITH installs AS (
    SELECT
        acquisition_channel,
        COUNT(DISTINCT device_code) AS install_count,
        SUM(cpi_uah) AS total_cpi
    FROM app_sessions
    GROUP BY acquisition_channel
),
revenue AS (
    SELECT
        app_sessions.acquisition_channel,
        SUM(orders_log.total_uah) AS total_revenue
    FROM app_sessions
    JOIN devices_users_map
        ON app_sessions.device_code = devices_users_map.device_code
    JOIN orders_log
        ON devices_users_map.user_uuid = orders_log.user_uuid
    GROUP BY app_sessions.acquisition_channel
)

SELECT
    installs.acquisition_channel,
    install_count,
    total_cpi,
    total_revenue,
    ROUND((total_revenue - total_cpi) / NULLIF(total_cpi, 0), 2) AS roi
FROM installs
LEFT JOIN revenue
    ON installs.acquisition_channel = revenue.acquisition_channel
ORDER BY roi DESC NULLS LAST;
"""

df_var_2 = pd.read_sql(text(query), engine)
df_var_2.head()

,acquisition_channel,install_count,total_cpi,total_revenue,roi
0,Organic,111,2576.55,937531.37,362.87
1,Facebook,134,3401.53,1150615.53,337.26
2,Referral,132,3369.93,994129.97,294.00
3,Google Ads,123,3196.90,913383.78,284.71


In [56]:
df

,acquisition_channel,installs,total_cost,avg_cpi,viewers,buyers,total_revenue
0,Facebook,134,3401.53,25.38,134,89,1150615.53
1,Referral,132,3369.93,25.53,132,73,994129.97
2,Organic,111,2576.55,23.21,111,70,937531.37
3,Google Ads,123,3196.90,25.99,123,68,913383.78


In [53]:
query = """
SELECT
acquisition_channel,
  SUM(cpi_uah)
FROM app_sessions
GROUP BY acquisition_channel
"""

df = pd.read_sql(text(query), engine)
df.head()

,acquisition_channel,sum
0,Organic,2576.55
1,Referral,3369.93
2,Facebook,3401.53
3,Google Ads,3196.90


### В чём **разница** между CTE и подзапросом?

|                    | Вложенный подзапрос (`SELECT (...)`)  | CTE (`WITH ...`)                            |
| ------------------ | ------------------------------------- | ------------------------------------------- |
| Пишется внутри     | Прямо в `SELECT`, `FROM`, `WHERE`     | В начале запроса                            |
| Уровень читаемости | Меньше, особенно если вложенность > 1 | Выше: структура запроса читается по шагам   |
| Переиспользуемость | Нет                                   | Да — можно использовать в нескольких местах |
| Многоэтапность     | Сложно реализовать                    | Удобно реализовать пошаговые расчёты        |
| Производительность | Обычно схожа                          | Может быть хуже или лучше — зависит от СУБД |

### Когда использовать:

| Хочешь...                                               | Используй |
| ------------------------------------------------------- | --------- |
| Вложить один простой `SELECT` внутрь `WHERE` или `FROM` | Подзапрос |
| Разделить сложный расчёт на понятные шаги               | CTE       |
| Использовать результат подзапроса в нескольких местах   | CTE       |
| Отладить или протестировать части логики отдельно       | CTE       |

---

> **CTE (WITH)** — это именованный подзапрос, вынесенный наверх и удобный для повторного использования и читаемости.  
> **Подзапрос** — это встроенный внутрь основного запроса кусок SQL-кода, без имени.

# Индексы и производительность SQL-запросов для аналитика

## Что такое индекс

**Индекс** — это дополнительная структура данных, которую СУБД (например, PostgreSQL, MySQL) создаёт для ускоренного поиска строк.

*Аналогия:* это как алфавитный указатель в книге: чтобы найти "Revenue Retention", ты не листаешь всю книгу, а идёшь в конец и находишь страницу по оглавлению.

## Как работает индекс

Когда ты делаешь запрос типа:

```sql
SELECT * FROM users WHERE email = 'oleg@example.com';
```

СУБД может:

* **без индекса** — просканировать всю таблицу (Full Table Scan),
* **с индексом** — найти `email` в индексе (обычно дерево) и перейти сразу к нужной строке.

🧠 Индекс содержит пары: `значение → ссылка на строку`.

## Как проверить, используется ли индекс (PostgreSQL)

```sql
EXPLAIN ANALYZE
SELECT * FROM users WHERE email = 'oleg@example.com';
```

Если используется индекс, ты увидишь строчку вида:

```
Index Scan using idx_users_email on users ...
```

Если нет — будет:

```
Seq Scan on users ...
```

📌 *Seq Scan* = полный перебор → медленно на больших таблицах.

---

## Когда индексы помогают

| Тип запроса              | Комментарий                 |
| ------------------------ | --------------------------- |
| `WHERE column = value`   | Самый эффективный случай    |
| `JOIN` по ключу          | Ускоряет соединение         |
| `ORDER BY created_at`    | Упорядочивание по индексу   |
| `GROUP BY channel`       | Если есть индекс по channel |
| `IN (...)` или `BETWEEN` | Может использовать индекс   |

## Когда индекс НЕ сработает

| Пример запроса                          | Почему не сработает                     |
| --------------------------------------- | --------------------------------------- |
| `WHERE LOWER(name) = 'oleg'`            | Индекс по `name`, а не по `LOWER(name)` |
| `WHERE price * quantity > 10000`        | Вычисление, индекс не применим          |
| `SELECT * FROM table`                   | Нет фильтра                             |
| `WHERE created_at::date = '2023-01-01'` | Приведение типов                        |

🧠 Индекс работает, только если выражение в `WHERE` **совпадает с тем, как создан индекс**.
Часто полезны индексы:

* по `user_id`, `order_id`, `device_code`
* по `created_at`, если часто фильтруете или сортируете по дате

## Как посмотреть существующие индексы (PostgreSQL)

```sql
SELECT
    tablename,
    indexname,
    indexdef
FROM pg_indexes
WHERE schemaname = 'public';
```

📌 Аналитик может использовать этот запрос, чтобы понять: *есть ли нужный индекс вообще*?

## Как выбрать, на что ставить индекс

**Шаг 1: Посмотри частые запросы**

* В `WHERE`, `JOIN`, `ORDER BY`, `GROUP BY`

**Шаг 2: Сделай EXPLAIN ANALYZE** — если `Seq Scan` на большой таблице — кандидат на индекс

**Шаг 3: Проверь дубли** — не дублируешь ли индекс по другим колонкам

## Побочные эффекты индексов

| Минус                            | Комментарий                               |
| -------------------------------- | ----------------------------------------- |
| Занимают место                   | Особенно составные индексы (`col1, col2`) |
| Замедляют `INSERT/UPDATE/DELETE` | Каждый индекс тоже нужно обновлять        |
| Могут быть избыточными           | Ухудшают производительность               |

📌 Если таблица — **log** или **events** — с миллионами записей в день, лучше не плодить лишние индексы.


## Практический пример: медленный запрос

Запрос:

```sql
SELECT COUNT(*)
FROM orders_log
WHERE user_uuid = 'abc123';
```

⛔ Работает медленно. EXPLAIN показывает:

```
Seq Scan on orders_log ...
```

📈 Оптимизация:

```sql
CREATE INDEX idx_orders_user_uuid ON orders_log(user_uuid);
```

Теперь:

```
Index Scan using idx_orders_user_uuid ...
```

## Как аналитик может ускорить работу:

* Перед тяжёлыми JOIN'ами проверь наличие индексов по `ключам соединения`
* Используй `EXPLAIN` для частых запросов (или `auto_explain`, если есть)
* Избегай функций в `WHERE` без нужных функциональных индексов
* Работай с DBA: предложи создать индекс или оптимизировать существующие


# Порядок выполнения SQL-запроса и почему это важно

## Порядок выполнения SQL-запроса  

SQL-запрос **не выполняется сверху вниз**, как его читают. СУБД интерпретирует его **по своему порядку**, логически.

| № | Этап             | Описание                        |
| - | ---------------- | ------------------------------- |
| 1 | `FROM`           | Источник данных, включая `JOIN` |
| 2 | `ON`             | Условие объединения при `JOIN`  |
| 3 | `WHERE`          | Фильтрация строк                |
| 4 | `GROUP BY`       | Группировка данных              |
| 5 | `HAVING`         | Фильтрация по агрегатам         |
| 6 | `SELECT`         | Выбор столбцов                  |
| 7 | `DISTINCT`       | Удаление дубликатов             |
| 8 | `ORDER BY`       | Сортировка                      |
| 9 | `LIMIT / OFFSET` | Ограничение количества строк    |

## Почему это важно?

### Алиасы (псевдонимы) не работают в `WHERE`

```sql
SELECT salary + bonus AS total_income
FROM employees
WHERE total_income > 50000; -- ❌ не сработает
```

Правильно:

```sql
SELECT salary + bonus AS total_income
FROM employees
WHERE salary + bonus > 50000;
```

`WHERE` идёт раньше `SELECT`, и он не знает про `total_income`

### `ORDER BY` использует алиасы

Здесь алиасы уже работают:

```sql
SELECT name, salary + bonus AS total_income
FROM employees
ORDER BY total_income DESC;
```

### Фильтрация по агрегатам — только в `HAVING`

```sql
SELECT department, SUM(salary) AS total_salary
FROM employees
GROUP BY department
HAVING SUM(salary) > 100000;
```

### JOIN сначала, фильтрация потом

```sql
SELECT *
FROM users
LEFT JOIN orders ON users.id = orders.user_id
WHERE orders.id IS NULL;
```
---

*  это «антиджойн»: ищем пользователей без заказов
*  **антиджойн** (anti-join) - способ найти строки **из одной таблицы**, которым **не соответствует ни одной строки** в другой таблице
*  `LEFT JOIN` берёт **всех пользователей** из таблицы `users`.
*  к каждому пользователю пытается «приклеить» заказ (`orders`), где `users.id = orders.user_id`.
*  если заказов нет — `orders.*` будет заполнено `NULL`'ами.
*  `WHERE orders.id IS NULL` оставляет **только те строки**, где **не найдено совпадений** в таблице `orders` — то есть где `orders.id IS NULL`.
*  мы **оставляем только тех пользователей**, у которых **нет заказов** — потому что заказ **не приклеился** во время `LEFT JOIN`.

Такой подход часто используется, когда нужно найти «висячие» или «осиротевшие» записи — например:

* Клиенты без покупок
* Товары без категории
* Комментарии без постов и т.д.

## Поддержка алиасов в SQL-диалектах

| SQL-диалект     | Алиас в `SELECT` можно использовать в `GROUP BY`? | Алиас в `SELECT` можно использовать в `ORDER BY`? | Комментарий                                  |
| --------------- | ------------------------------------------------- | ------------------------------------------------- | -------------------------------------------- |
| **PostgreSQL**  | ✅ Да                                              | ✅ Да                                              | Очень гибкая                                 |
| **MySQL ≥ 8.0** | ❌ Нет                                             | ✅ Да                                              | В `GROUP BY` нужно писать выражение          |
| **MySQL < 8.0** | ❌ Нет                                             | ✅ Да                                              | Старые версии                                |
| **SQLite**      | ❌ Нет                                             | ✅ Да                                              | Только `ORDER BY` видит алиас                |
| **SQL Server**  | ❌ Нет                                             | ✅ Да                                              | `GROUP BY` требует исходное выражение        |
| **Oracle**      | ❌ Нет                                             | ✅ Да                                              | `GROUP BY` не поддерживает алиасы            |
| **BigQuery**    | ✅ Да                                              | ✅ Да                                              | Полная поддержка                             |

# Слияние данных из разных источников

## **паркетный файл (.parquet)**

**Parquet** — это *колоночный* формат хранения данных, используемый для эффективной работы с большими таблицами. Он особенно популярен в анализе данных, машинном обучении и хранилищах данных.

## В чем отличие от CSV?

| Характеристика         | CSV                           | Parquet                                  |
| ---------------------- | ----------------------------- | ---------------------------------------- |
| Формат                 | Текстовый                     | Бинарный, колоночный                     |
| Размер файла           | Большой                       | Меньше (в 2–10 раз сжатие)               |
| Скорость чтения        | Медленнее                     | Быстрее при выборке по столбцам          |
| Чтение частично        | Нет                           | Да (можно выбрать только нужные столбцы) |
| Поддержка типов данных | Нет (все строки)              | Да (int, float, datetime и т.д.)         |
| Удобство для глазами   | Да (можно открыть в блокноте) | Нет (бинарный формат)                    |

---

## Почему "колоночный"?

CSV и Excel — **построчные** (row-oriented): данные читаются строка за строкой.

Parquet — **поколоночный** (column-oriented): данные читаются по столбцам. Это даёт преимущества:

* Можно читать только нужные столбцы (экономия I/O).
* Лучше сжатие (однотипные данные в столбце).
* Быстрее агрегации и фильтрации по колонке.

---

## Где используется Parquet?

| Сфера                     | Пример использования                                 |
| ------------------------- | ---------------------------------------------------- |
| Облачные хранилища данных | Amazon S3, Google Cloud Storage + BigQuery           |
| Аналитика/ETL             | Apache Spark, Dask, Presto, Pandas                   |
| Машинное обучение         | Подготовка признаков (features) для моделей          |
| Хранилища данных (DWH)    | Паркет — стандарт де-факто для lakehouse-архитектуры |

---

## Пример: когда использовать Parquet

Ты выгружаешь 10 млн строк о заказах клиентов. Если это CSV — вес может быть 1 ГБ. В формате Parquet — 100–300 МБ. Если тебе нужно посчитать средний чек — ты можешь прочитать **только** столбец с суммой покупки.

---

## Как сохранить и прочитать в Pandas:

```python
df.to_parquet("data.parquet")       # сохранить
df = pd.read_parquet("data.parquet")  # прочитать
```

> Для работы с `.parquet` нужны библиотеки: `pyarrow` или `fastparquet`.

Установить можно так:

```bash
pip install pyarrow
```

## Пример

In [ ]:
# Get Data from DB


In [ ]:
# Get Data from Parquet


In [ ]:
# INNER JOIN (только совпадения)


In [ ]:
# LEFT JOIN (все заказы, даже без информации о пользователе)


In [ ]:
# RIGHT JOIN (все пользователи, даже без заказов)


In [ ]:
# FULL OUTER JOIN (все данные: и заказы, и пользователи)
